In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
import os
# check the current path
print(os.getcwd()) # /content

# You should copy the path : 왼쪽 폴더에서 오른쪽 버튼 후 경로복사
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW')

print(os.getcwd()) # path has been changed

In [ ]:
# 라이브러리 설치
!pip install efficientnet-pytorch
!pip install albumentations
!pip install timm
!pip install scikit-learn matplotlib

In [ ]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install facenet-pytorch

# 2. 데이터 로드 및 처리

In [ ]:
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

input_dirs = [
    "/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/dfdc_train_part_0",
    "/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/dfdc_train_part_1"
]

dfs = []

for input_dir in input_dirs:
    json_file = os.path.join(input_dir, "metadata.json")
    with open(json_file, "r") as f:
        metadata = json.load(f)

    df = pd.DataFrame.from_dict(metadata, orient="index").reset_index()
    df.columns = ["filename", "label", "split", "original"]
    df["label"] = df["label"].map({"FAKE": 0, "REAL": 1})
    dfs.append(df)

# 데이터셋 합치기
df_combined = pd.concat(dfs, ignore_index=True)

# 라벨 통계 출력
real_count = (df_combined["label"] == 1).sum()
fake_count = (df_combined["label"] == 0).sum()

print(f"REAL videos: {real_count}")
print(f"FAKE videos: {fake_count}")

# 데이터 균형 유지 및 Train/Test Split
df_label_1 = df_combined[df_combined["label"] == 1]
df_label_0 = df_combined[df_combined["label"] == 0]

n_samples = len(df_label_1)
df_label_0_sampled = resample(df_label_0, replace=False, n_samples=n_samples, random_state=42)

df_balanced = pd.concat([df_label_1, df_label_0_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

train_df, test_df = train_test_split(df_balanced, test_size=0.2, random_state=42)

# 3. 데이터셋 클래스 및 DataLoader

In [ ]:
import cv2
from facenet_pytorch import MTCNN
from PIL import Image
from tqdm import tqdm

# MTCNN 모델 초기화
mtcnn = MTCNN(keep_all=False)

# 처리된 비디오 출력 경로
output_dir = "/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/all2_2_processed_videos"
os.makedirs(output_dir, exist_ok=True)

skipped_videos = []

# 비디오 처리 함수
def process_video(video_info):
    video_path, output_path = video_info
    if not os.path.exists(video_path):
        return output_path

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Skipping invalid video: {video_path}")
        skipped_videos.append(video_path)
        return None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        box, _ = mtcnn.detect(Image.fromarray(rgb_frame))
        if box is not None:
            x1, y1, x2, y2 = map(int, box[0])
            if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
                continue
            cropped_face = frame[y1:y2, x1:x2]
            cropped_face_resized = cv2.resize(cropped_face, (224, 224))
            cv2.imwrite(output_path, cropped_face_resized)
            break

    cap.release()
    return output_path

# 모든 데이터셋 처리
for input_dir in input_dirs:
    video_info_list = [
        (os.path.join(input_dir, filename), os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.jpg"))
        for filename in os.listdir(input_dir) if filename.endswith(".mp4")
    ]
    print(f"Processing videos in {input_dir}...")
    for video_info in tqdm(video_info_list):
        process_video(video_info)
print("Processing complete.")

# 4. 모델 정의

In [ ]:
from timm import create_model
import torch.nn as nn

class CustomEfficientNetB0(nn.Module):
    def __init__(self, pretrained=True, num_classes=1):
        super(CustomEfficientNetB0, self).__init__()
        self.backbone = create_model("efficientnet_b0", pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.backbone.forward_features(x)
        x = self.dropout(x)
        x = x.mean(dim=[2, 3])
        x = self.fc(x)
        return x

class CustomConvNextTiny(nn.Module):
    def __init__(self, pretrained=True, num_classes=1):
        super(CustomConvNextTiny, self).__init__()
        self.backbone = create_model("convnext_tiny", pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, x):
        x = self.backbone.forward_features(x)
        x = self.dropout(x)
        x = x.mean(dim=[2, 3])
        x = self.fc(x)
        return x

# 5. 학습 및 테스트 함수 정의

In [ ]:
import torch.optim as optim
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from timm import create_model
from tqdm import tqdm

def train(model, optimizer, scheduler, train_loader, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device).unsqueeze(1).float()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}")
        scheduler.step(epoch_loss)

def test_model(model, test_loader, device):
    model.eval()
    y_true, y_pred = [], []
    test_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).unsqueeze(1).float()
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            predictions = torch.sigmoid(outputs).cpu().numpy() > 0.5
            y_pred.extend(predictions.flatten())
            y_true.extend(labels.cpu().numpy().flatten())

    accuracy = sum(1 for a, b in zip(y_true, y_pred) if a == b) / len(y_true) * 100
    return {"Test Loss": test_loss / len(test_loader), "Accuracy": accuracy}, y_true, y_pred

# 6. 모델 학습 및 테스트

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

class DeepFakeDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, f"{os.path.splitext(self.df.iloc[idx]['filename'])[0]}.jpg")
        image = Image.open(image_path).convert('RGB')
        label = self.df.iloc[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, label

# 처리된 비디오 출력 경로
output_dir = "/content/gdrive/MyDrive/Colab Notebooks/2024_OSP_SW/all2_processed_videos"

# 'cell 3'에서 처리된 이미지가 저장된 디렉토리로 설정
image_dir = output_dir

# 이미지 변환을 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터셋을 생성
train_dataset = DeepFakeDataset(train_df, image_dir, transform)
test_dataset = DeepFakeDataset(test_df, image_dir, transform)

# 데이터 로더를 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # 배치 크기 조정하며 변경
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) # 배치 크기 조정하며 변경

# 사용할 장치를 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
criterion = nn.BCEWithLogitsLoss()

efficientnet_b0 = CustomEfficientNetB0()
optimizer_eff = optim.Adam(efficientnet_b0.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler_eff = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_eff, mode="min", factor=0.5, patience=2)

print("Training EfficientNet-B0...")
train(efficientnet_b0, optimizer_eff, scheduler_eff, train_loader, num_epochs=10)

print("\nTesting EfficientNet-B0...")
metrics_eff, y_true_eff, y_pred_eff = test_model(efficientnet_b0, test_loader, device)
print(metrics_eff)

convnext_tiny = CustomConvNextTiny()
optimizer_conv = optim.Adam(convnext_tiny.parameters(), lr=5e-6, weight_decay=1e-4)
scheduler_conv = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_conv, mode="min", factor=0.5, patience=2)

print("Training ConvNextTiny...")
train(convnext_tiny, optimizer_conv, scheduler_conv, train_loader, num_epochs=10)

print("\nTesting ConvNextTiny...")
metrics_conv, y_true_conv, y_pred_conv = test_model(convnext_tiny, test_loader, device)
print(metrics_conv)

# 7. 혼동 행렬 시각화
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred, class_names=["FAKE", "REAL"]):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

plot_confusion_matrix(y_true_eff, y_pred_eff)
plot_confusion_matrix(y_true_conv, y_pred_conv)